In [27]:
"""This section creates the ini files for VRT."""
import os

project_path = 'D:/ERCOT/AEP Dynamic/MQT/Projects/'
project_list = os.listdir(project_path)

vrt_path = 'D:/DMView 3.3/'

for project in project_list:
    scenario = ['dis','ch']
    initials = ['lag','lead']
    for bess in scenario:
        for initial in initials:
            """Create the ini file."""
            filename = vrt_path + project + '_VRT_' + bess + '_' + initial + '.ini'

            """Write the ini file."""
            with open(filename, 'w') as ini:
                ini.write('/' + project + 'for ERCOT dynamic model quality test\n')
                ini.write("/created by Arthur Li - CF Power\n\n")
                ini.write("[Build FS]\nBuild_FS_flag = 1\nPOI_PF = ")
                if initial == 'lag':
                    ini.write('0.95 /lagging \n')
                else:
                    ini.write('-0.95 /leading \n')

                ini.write('\n[Input files]\ninput_path = CASEs\\'+project+'\n')
                ini.write('unconv_casefile ='+project+ '_'+ bess+'.sav /psse v33\n model_file_lst = [\''+project+'.dyr\']/codes in v33\n')
                ini.write('\n[Tests]\nTest01_LVRT_LEGACY = [\'VOLT\',\'DATAs\\ERCOT_Legacy_LVRT.xlsx\']\n')
                ini.write('Test02_LVRT_VOLT_DIP = [\'VOLT\',\'DATAs\\ERCOT-245_2800_Piecewise_LVRT_(PV-BESS)_10secSpacing_r2.xlsx\']\n')
                ini.write('Test03_HVRT_PREFERED = [\'VOLT\',\'DATAs\\ERCOT-245_2800_HVRT.xlsx\']\n\n[Settings]\nPlot_Flag = 1')

In [ ]:
"""This section runs the VRT."""
os.chdir(vrt_path)

#for project in project_list:
project = '24INR0442 MIDNIGHT SUN ENERGY STORAGE'
for bess in scenario:
    for initial in initials:
        filename = vrt_path + project + '_VRT_' + bess + '_' + initial + '.ini'
        os.system('python dmview.py "'+filename+'"')
print('Finish the VRT of '+project+' successfully.\n')


In [ ]:
"""This section plots the VRT results."""
import os
import sys
import psse35
import psspy
import dyntools
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
import glob

sys_path_PSSE = r'C:\Program Files\PTI\PSSE35\35.5\PSSPY39'  # or where else you find the psspy.pyc
sys.path.append(sys_path_PSSE)

os_path_PSSE = r'C:\Program Files\PTI\PSSE35\35.5\PSSBIN'  # or where else you find the psse.exe
sys.path.append(os_path_PSSE)
os.environ['PATH'] += ';' + os_path_PSSE
os.environ['PATH'] += ';' + sys_path_PSSE

# Create list with all .OUT files in directory
name = '24INR0442 MIDNIGHT SUN ENERGY STORAGE_VRT_DIS_LEAD'    # Change the test name here
outpath = 'D:\DMView 3.3\RESULTs\\'+name
outlist = []
tests = ['TEST01_LVRT_LEGACY_VOLT','TEST02_LVRT_VOLT_DIP_VOLT','TEST03_HVRT_PREFERED_VOLT']
#tests = ['TEST00_FS_FS']
for test in tests:
    out = glob.glob(outpath+'\\'+test+'\*.out')[0]
    outlist.append(out)

    outfile = dyntools.CHNF(out)

    short_title, chanid_dict, chandata_dict = outfile.get_data()

    # Plot VRT result
    # p_num, q_num, v_num = 9, 16, 81 # Short Creek
    # p_num, q_num, v_num = 3, 6, 22  # Solara
    p_num, q_num, v_num = 6, 10, 62  # Midnight sun
    time = chandata_dict['time']
    title = name+':'+test
    labels = ['Real Power','Reactive Power','Voltage']

    fig, (ax1, ax2) = plt.subplots(2, sharex=True)
    fig.suptitle(title)

    ax1.set_ylabel('Real and Reactive Power (pu)')
    ax1.plot(time, chandata_dict[p_num], color='tab:green',label='Real Power')
    ax1.plot(time, chandata_dict[q_num], color='tab:red',label='Reactive Power')
    ax1.grid(linestyle = '--')

    ax2.set_xlabel('Time (seconds)')
    ax2.set_ylabel('Voltage (pu)')
    ax2.plot(time, chandata_dict[v_num], color='tab:blue',label='Voltage')
    ax2.grid(linestyle = '--')

    fig.tight_layout()  # otherwise the right y-label is slightly clipped
    if test == 'TEST03_HVRT_PREFERED_VOLT':
        plt.xlim(0,20)
    elif test == 'TEST02_LVRT_VOLT_DIP_VOLT':
        plt.xlim(0,60)
    else:
        plt.xlim(0,25)
    fig.legend([ax1, ax2], labels=labels, loc = 'center right') 
    plt.show()